In [ ]:
# Mount google drive, install rasterio
import os, numpy as np, pandas as pd, random, sys
modeltype = "unet"
dataname = "lang"
path = "/content/drive/My Drive/jöklar/"
sys.path.append(path + "src")
from util.util import install_import
install_import("rasterio")
import rasterio
model_path = path + "results/" + modeltype + "/"
data_path = path + "data/" + dataname + "/"
os.makedirs(model_path, exist_ok=True)
os.chdir(model_path)

In [ ]:
# Read image data
%%time
(img, mask, is_on_border, train, val, test) = (np.load(data_path +
                                              "images.npz").values())
X = img[is_on_border].astype(np.float32)/10000
Y = mask[is_on_border]

In [ ]:
# Read training history, best model and model predictions
from tensorflow.keras.models import load_model
history = pd.read_csv('result_history.csv')
model = load_model('model_first.keras')
model.load_weights('model_best.keras')
print(history[-3:])
loaded = np.load("probs.npz")
(probs, test_loss, test_accuracy) = loaded.values()
train_probs = probs[train]
val_probs = probs[val]
test_probs = probs[test]
print(f"Test accuracy: {test_accuracy}")

In [ ]:
# Plot training history
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 3))
plt.title("Learning curve")
plt.plot(history.loss, label="loss")
plt.plot(history.val_loss, label="val_loss")
plt.plot(np.argmin(history.val_loss), history.val_loss.min(), marker="x",
         color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
# Plot probabilities according to data split
from matplotlib.ticker import PercentFormatter
fig, axes = plt.subplots(1, 3, figsize=(8, 2), sharey=True)
datasets = [train_probs, val_probs, test_probs]
labels = ['Train', 'Validation', 'Test']
for i, (data, label) in enumerate(zip(datasets, labels)):
    axes[i].hist(data.ravel(), bins=10, range=(0, 1), rwidth=0.8, color='skyblue',
                 ec='k', zorder=3, density=True)
    axes[i].set_xlabel('Probabilities')
    axes[i].set_ylabel('Percentage' if i == 0 else '')
    axes[i].set_title(label)
    axes[i].grid(True, axis='y', linestyle='-', lw=1, color='lightgray', zorder=0)
    #if i==0:
    axes[i].yaxis.set_major_formatter(PercentFormatter(10, decimals=0))
plt.tight_layout()
plt.show()

In [ ]:
def plot_sample(X, y, preds, binary_preds, indices):
    """Function to plot the results"""
    nrows = len(indices):
    print(ix)
    for (irow, ix) in enumerate(indices):
      has_mask = y[ix].max() > 0
      fig, ax = plt.subplots(nrows, 4, figsize=(12,3))

      ax[irow].imshow(X[ix, ..., 0])
      if has_mask:
          ax[irow].contour(y[ix].squeeze(), colors='k', levels=[0.5])
      if ix == indices[0]:
        ax[i].set_title('Original')

        ax[1].imshow(y[ix].squeeze(), cmap = 'gray')
        ax[1].set_title('Mask')

        ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1, cmap = 'gray')
        if has_mask:
            ax[2].contour(y[ix].squeeze(), colors='r', levels=[0.5])
        ax[2].set_title('Glaciers Predicted')

        ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1, cmap = 'gray')
        if has_mask:
            ax[3].contour(y[ix].squeeze(), colors='r', levels=[0.5])
        ax[3].set_title('Glaciers Predicted binary');
# Check if training data looks all right
train_probs_01 = (train_probs > 0.5).astype(np.uint8)
plot_sample(img[train], mask[train], train_probs, train_probs_01, 69)

In [ ]:
# Check if validation data looks all right
plot_sample(img[val], mask[val], val_probs, val_probs_t)

In [ ]:
# Predict each image and write the output as mask raster
# Mýr
# n_images = 115
# Lan
import rasterio, json
from cnn_util.meta_util import dejsonify_list
n_images = len(img)
n_channels = 13
threshold = 0.5
with open('meta.json') as f:
    meta_list = dejsonify_list(json.load(f))
print(os.getcwd())
for i in range(n_images):
  f = rasterio.open("/content/drive/MyDrive/tif_img_256_lan_more/img_" + str(i) + ".tif")
  prediction_t = (probs[i] > threshold).astype(np.uint8)
  meta = meta_list[i]
  meta["nodata"] = 0
  meta["count"] = 1
  new_arr = np.rollaxis(prediction_t[:,:,:], 2, 0)
  filename = "test" + str(i) + ".tif"
  print(filename)
  with rasterio.open(filename, "w", **meta) as dest:
    dest.write(new_arr)

In [ ]:
os.getcwd()

# **Búa til nýjar útlínur**

In [ ]:
img.shape

In [ ]:
import glob
from osgeo import gdal
import subprocess

demList = glob.glob("test*.tif")
print(len(demList))

In [ ]:
vrt = gdal.BuildVRT("merged.vrt", demList)
gdal.Translate("mergedDEM-deeplab-mobilenet-myr.tif", vrt, xRes=10, yRes=-10)
vrt = None

In [ ]:
from rasterio.plot import show
r = rasterio.open("mergedDEM-deeplab-mobilenet-myr.tif")
show(r)

In [ ]:
!gdal_polygonize.py mergedDEM-deeplab-mobilenet-myr.tif -f "ESRI Shapefile" mergedDEM-deeplab-mobilenet-myr.shp

In [ ]:
import geopandas as gpd
stat = gpd.read_file("mergedDEM-deeplab-mobilenet-myr.shp")
stat.plot()